<p style="text-align:center;font-size:small;font-style:italic;">Disclaimer: I don't remember last time I've wrote any project as fast as this one. I learned about competition less than 48h ago, started coding and exploring data only next day. There is so many things one can do with Kaggle Survey. There is so many things I didn't have time to do. Hopefully, other kagglers will venture were I couldn't. Therefore, this notebook will seem to 'lack', not for a 'lack' of trying, but simply because data analysis is always time consuming craft and without time to spare we need to make compromises. This also means that I didn't manage to re-write my code into separate functions or clean it with duplicate operations. You've been warned:)</p>

<h1 style="color:blue;text-align:center;">Exploring Macro Outlook of Kaggle ML Survey with a Hint of Trends Exploration from The Past</h1>

<p style="text-align:center;">Following notebook is divided in two parts. This is partially because of lack of time and partially because I wanted to explore so much that I started to take jabs at everything. Take me at my word, this survey is extraordinary dataset! To start with, we have **19718** participants. It means that we have representable sample data and all of derived insights can be taken only with a small grain of salt, just enough to account for selection bias. I also feel responsible to mention a word or two about methodology. Part I mostly focus on macro data. We explore **age, gender, country, education and salary questions**. This served both as a 'warm up' before diving deeper into dataset and to show how big, diverse and also unequal reality of DS can be. **Part I doesn't require much of an afterthought, it's simple quantitative data**. Part II is exploration of a past. **I decided to include previous (2018) survey into this analysis**. Why not 2017? Dataset from 2017, interesting as it is, seems to be structured in a very different form. Even survey from past year required considerable amount of 'fitting', but survey from 2017 seems to happen in a little different reality for Kaggle Team *(yay for very specific selection options!)*. This illustrates few very well known points, **a) most of data science jobs is actually data cleaning job, b) data fitting means data selection c) data selection most likely means bias**. Yes, my data is biased! When comparing past Kaggle surveys you can easily notice a pattern of increased precision with questions. This precision lead to impossibility of fitting data exactly as I wanted for comparison operations. So there are certain trade offs. But, don't worry, you can always look into code (advised) or read explanation of 'what exactly happened here' carefully. We should be good. </p>

<h3 style="text-align:center;">Key points</h3>
<ul style='background-color:lightgrey;text-align:center;'>
<li>1. Learn about general macro landscape of data science industry through analysis of Kaggle ML 2019 Survey.</li>
<li>2. Find patterns in relations between different vectors of data (like income/country).</li>
<li>3. Provide industry insight.</li>
<li>4. Check changes between 2018 and 2019 survey in data science and machine learning.</li></ul>


<h3 style='text-align:center;color:green'>Table of contents</h3>
<ul style='background-color:lightblue;text-align:center;'><li>1. Exploring Age/Gender Relation</li>
<li>2. Exploring Income/Country Relation</li>
<li>3. Exploring Education/Company Size Relation</li>
<li>4. Exploring Education/Income Relation</li>
<li>5. Exploring Years Coding/Years using ML Relation</li>
<p style="font-style:italic;text-align:center;">Further points will start digging into past data to see trend changes on relevant questions<p>
<li>6. Favorite Media Trends through out 2018/2019</li>
<li>7. Favorite ML Tools through out 2018/2019</li></ul>


<h3 style="text-align:center;">Import libraries, loading data, basic preprocessing</h3>
<p style="text-align:center;">I selected only part of dataframe to operate on. This will be used to explore all of macro questions.  I also avoided changing names of columns, actually, operating on shorter column names, like 'Q10', is easier than long but more descriptive ones *(We have schema to lookup. Imagine changing names in terabyte sized BQ tables.)*. There is also interesting pattern to be found (and explored) in previous Kaggle Surveys. Namely, **each year surveys get more and more specific, both in their schema as well as defined questions**. Following Kaggle team approach to dataset (of increased precision), I will operate on what is provided and only rename data when I actually use it.</p>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from collections import OrderedDict
import numpy as np

multiple_choice = pd.read_csv('../input/kaggle-survey-2019/multiple_choice_responses.csv')
multiple_choice_2018 = pd.read_csv('../input/kaggle-survey-2018/multipleChoiceResponses.csv')
# other_text_responses = pd.read_csv('../input/kaggle-survey-2019/other_text_responses.csv')
# questions_only = pd.read_csv('../input/kaggle-survey-2019/questions_only.csv')
# survey_schema = pd.read_csv('../input/kaggle-survey-2019/survey_schema.csv')

personal = multiple_choice[['Q1', 'Q2', 'Q3', 'Q4', 'Q6', 'Q10']]

<h1 style="text-align:center;color:blue;">Exploring Age/Gender Relation</h1>

<p style="text-align:center;">Age/Gender relations are usually unsuprising in computer science, this is no different in case of Kaggle survey. However, every data exploration needs some start, a plot which represents some basic information from our dataset, a plot which functions as motivation for next, more challenging one. This is such plot.</p>

<ul style="text-align:center;">
    <li style="background-color:lightblue">Male                       16138</li>
    <li style="background-color:lightpink">Female                      3212</li></ul>



In [ ]:
age_gender = personal.groupby(['Q2', 'Q1']).size().rename('count').reset_index()
age_gender.columns = ['gender', 'age', 'count']
age_gender.drop(age_gender.index[44], inplace=True)
age_gender_viz = (age_gender.pivot_table(index='age', columns='gender', values='count'))
ax = age_gender_viz.plot(kind='barh', width=1.0, colors=['r', 'b', 'g', 'm'], alpha=0.3, title='Age/Gender Relation')
ax.set_xlabel('No. of responders')
ax.set_ylabel('Age')

<h1 style="text-align:center;color:blue;">Exploring Income/Country Relation</h1>
<p style="text-align:center;">I think this is most interesting insight of all. And my hunch is that this is because of scatter plot which shows such relations better than bars. I selected top 10 **(by count of votes)** countries of origin and plotted them against claimed compensation. What we see is big disparity between regions. If you pay attention you can notice 3 distinctive clusters *(actually it's four, one in the lower left corner can be treated as separate cluster)* formed around USA, India and Europe. It's clearly visible that US has best wages for data scientist, but it's amazing HOW MUCH better are they comparing to India which has same amount of workers **(1310 USA vs 1349 India)**. Another cluster, formed around Europe and Brazil/Japan/Canada. Here we can see that income inequality is shared also among countries with smaller workforce. Russia and Brazil are well below other European salaries. Notice also that Canadian incomes are somehow closer to US, even when amount of people claiming Canada as their country is small.</p>

In [ ]:
salaries = personal.groupby(['Q3', 'Q10']).size().rename('count').reset_index()
salaries_count = salaries.groupby('Q3').sum()
salaries_reported = salaries_count['count'].nlargest(10)
salaries.set_index('Q3', inplace=True)
top10 = salaries.loc[['India', 'United States of America', 'Brazil', 'Japan', 'Russia', 'Germany', 'United Kingdom of Great Britain and Northern Ireland', 'Spain', 'Canada']]

india = top10.loc['India'].sort_values(by='count').nlargest(columns='count', n=5).reset_index().set_index('Q10')
usa = top10.loc['United States of America'].sort_values(by='count').nlargest(columns='count', n=5).reset_index().set_index('Q10')
brazil = top10.loc['Brazil'].sort_values(by='count').nlargest(columns='count', n=5).reset_index().set_index('Q10')
japan = top10.loc['Japan'].sort_values(by='count').nlargest(columns='count', n=5).reset_index().set_index('Q10')
russia = top10.loc['Russia'].sort_values(by='count').nlargest(columns='count', n=5).reset_index().set_index('Q10')
germany = top10.loc['Germany'].sort_values(by='count').nlargest(columns='count', n=5).reset_index().set_index('Q10')
uk = top10.loc['United Kingdom of Great Britain and Northern Ireland'].sort_values(by='count').nlargest(columns='count', n=5).reset_index().set_index('Q10')
spain = top10.loc['Spain'].sort_values(by='count').nlargest(columns='count', n=5).reset_index().set_index('Q10')
canada = top10.loc['Canada'].sort_values(by='count').nlargest(columns='count', n=5).reset_index().set_index('Q10')

top_df = pd.concat([india, usa, brazil, japan, russia, germany, uk, spain, canada])
top_df.reset_index(inplace=True)
top_df.Q10.str.replace('$', '')
top_df.Q10 = top_df.Q10.str.replace(r'\d+,\d+-', '')
top_df.Q10 = top_df.Q10.str.replace(r'\d+-', '')
top_df.Q10 = top_df.Q10.str.replace(r'\W', '')
top_df.Q3 = top_df.Q3.str.replace('United Kingdom of Great Britain and Northern Ireland', 'UK')
top_df.Q3 = top_df.Q3.str.replace('United States of America', 'USA')
top_df.Q10 = top_df.Q10.astype(int)
top_df['count'] = top_df['count'].astype(int)

fig, ax = plt.subplots()
nrof_labels = len(top_df['Q3'])
colors = cm.rainbow(np.linspace(0, 1, nrof_labels))

for i, c in enumerate(colors):
    x = top_df['count'][i]
    y = top_df['Q10'][i]
    l = top_df['Q3'][i]
    ax.scatter(x, y, label=l, c=c, alpha=0.5)

handles, labels = plt.gca().get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())
ax.set_xlabel('No. of people claiming given salary')
ax.set_ylabel('Salary in USD (max value)')
ax.set_title('Income/Country Relation')
plt.show()

<h1 style="text-align:center;color:blue;">Exploring Education/Company Size Relation</h1>
<p style="text-align:center;">I didn't want to check salary/education relation, so I opted for checking education/size of company relation. Reason? It matters more. Salaries are usually fairly standardized across given industry. Yes, you can be super star data scientist or successful CEO running data company and skew my bars so much for me to mistake it as insight, but that's outlier case. Education relation to size of company in which person works is far more interesting. It seems less arbitrary and independent of other conditions. I wasn't disappointed. **Plot clearly shows that people with PhD, same as people without any degree clearly prefer to work in small companies**. **My take here is that self-study leads mostly to finding jobs in aspiring startups, while PhD leads mostly to funding some early venture (think biotech).**</p>

In [ ]:
edu_plot = pd.Series(personal.Q4.value_counts())
edu_plot.drop(labels='What is the highest level of formal education that you have attained or plan to attain within the next 2 years?', inplace=True)
work_plot = pd.Series(personal.Q6.value_counts())
work_plot.drop(labels='What is the size of the company where you are employed?', inplace=True)

edus = personal.groupby(['Q4', 'Q6']).size().rename('count')
edus_df = pd.DataFrame(edus)
edus_df.drop(index=edus_df.index[-1:], inplace=True)
edus_df.reset_index(inplace=True)
edus_df.Q4 = edus_df.Q4.str.replace('Some college/university study without earning a bachelor’s degree', 'Some study without earning degree')
edus_df.columns = ['Education', 'Size of Company', 'Count']
edus_df['Size of Company'] = edus_df['Size of Company'].str.replace('10,000','10000')
edus_df['Size of Company'] = edus_df['Size of Company'].str.replace('9,999','9999')
edus_df.set_index(['Education', 'Size of Company'], inplace=True)
ax = edus_df.unstack().plot(kind='barh', alpha=0.3)
ax.set_xlabel('No. of Responses')
ax.set_ylabel('Level of Education')
ax.set_title('Education/Size of Company Employer Company')
handles, labels = ax.get_legend_handles_labels()
labels_new = [label.strip('()').split(',')[1] for label in labels]
plt.legend(handles, labels_new)

<h1 style="text-align:center;color:blue;">Exploring Years Coding/Years using ML Relation</h1>
<p style="text-align:center;">Here I plot answers specifying programming experience in years against years where person uses ML methods in her/his workflow. Pretty standard. But, it seems that people with less than 1 year and between 3-5 years of experience of coding tend to treat ML as basic programming skill *(they use ML methods from beginning)*. Of course, this is nice example of dataset bias, we are using survey within ML community, all of 'novice' answers will be skewed towards stating that they are using ML from beginning of their programming journey. However, after inspecting other bars which represent more experienced group of programmers (+5 years) you can also clearly see that they've begun using ML also only few years ago. This confirms sort of derogatory nevertheless true 'data bubble' term used so often in media.</p>
<p style="text-align:center;font-size:small;">Disclaimer: Here, I needed to merge few 'years brackets' from question how long ML methods are used into one (3-5 years). It means that we are looking at section 3-5 years composed of 3 separate (2-3yr ,3-4yr ,4-5yr) 'year brackets' and summed up into one.</p>

In [ ]:
code_long = multiple_choice[['Q15', 'Q23']]
code_long.drop(code_long[0:1].index, inplace=True)
code_long.dropna(inplace=True)
q15_values = pd.DataFrame(code_long['Q15'].value_counts())
q23_values = pd.DataFrame(code_long['Q23'].value_counts())
entry = pd.DataFrame(q23_values.loc[['2-3 years', '3-4 years', '4-5 years']].sum())


q23_values.drop(['2-3 years', '3-4 years',], inplace=True)
q23_values.rename(index={'4-5 years' : '3-5 years', '10-15 years': '10-20 years'}, inplace=True)
q23_values.Q23.replace(927, 3847, inplace=True)
q23_values.columns = ['ML Methods']
q15_values.columns = ['Years Programming']
ax = pd.concat({'Years Coding':q15_values, 'Years Using ML Methods': q23_values}, axis=1).plot.barh(alpha=0.3, color=['g','b'])
ax.set_xlabel('No. of Answers')
ax.set_ylabel('Years Coding/Years Using ML Methods')
ax.set_title('Relation of experience (in years) in programming to years using ML methods')
handles, labels = ax.get_legend_handles_labels()
labels_new = [label.strip('()').split(',')[1] for label in labels]
plt.legend(handles, labels_new)
plt.show()

<h1 style="text-align:center;color:blue;">Do we use different media in 2019 comparing to 2018? Different how?</h1>
<p style="text-align:center;">I promised bias at the beginning so bias I deliver. It was very interesting for me to have oportunity to see how information sources for ML community evolved, but, previous Kaggle surveys had very different selection options than current one. Actually, today we get fairly 'overall' options to pick from, but in 2018 we were asked about very specific things, ie. [FiveThirtyEight.com](http://FiveThirtyEight.com) which somehow didn't make a cut this year. This means that I needed to grab good old fashioned pen and paper and try to figure out how to fit two distinctive sets of answers into one using some common key. This had it's own issue. First - If I start to model answers to much I can destroy whatever information I wanto pass, that's bad, very bad. I allowed myself to do so only because question was very interesting and answering it without using 2 year timeline would loose a lot of insight. I compromised on fitting categories which, without a doubt, could be fitted by most of people. I left rest 'as is'. This means that we get someway 'ugly' but informative plot. Without doing so it would be impossible to notice that Kaggle didn't ask about Slack & Reddit in 2018 survey but is pretty consistent with asking about Blogs, Scientific Journals and Kaggle Forum. Also, it seems to be interest in our Youtube choices *(no question about youtube channels in 2019 thou!)*.</p>

In [ ]:
favorite_media_df = multiple_choice.filter(regex='Q12_')
fav_count = favorite_media_df.apply(lambda x: x.value_counts())
fav_count.drop(fav_count[11:].index, inplace=True)
fav_count.rename(index={'Kaggle (forums, blog, social media, etc)':'Kaggle forums',
                        'Blogs (Towards Data Science, Medium, Analytics Vidhya, KDnuggets etc)':'Blogs',
                        'Journal Publications (traditional publications, preprint journals, etc)':'Journal Publications',
                        'Course Forums (forums.fast.ai, etc)': 'Course Forums',
                        'Twitter (data science influencers)': 'Twitter',
                        'Reddit (r/machinelearning, r/datascience, etc)': 'Social/Reddit',
                        'Slack Communities (ods.ai, kagglenoobs, etc)': 'Social/Slack',
                        'Podcasts (Chai Time Data Science, Linear Digressions, etc)': 'Podcasts',
                        'Hacker News (https://news.ycombinator.com/)': 'Hacker News'
                        }, inplace=True)
fav_count = fav_count.filter(regex='Q12').bfill(axis=1).iloc[:, 0].fillna('unknown')
fav_count.sort_values(inplace=True)

favorite_media_df_2018 = multiple_choice_2018.filter(regex='Q38')
fav_count_2018 = favorite_media_df_2018.apply(lambda x: x.value_counts())
fav_count_2018.drop(fav_count_2018[22:].index, inplace=True)
fav_count_2018 = fav_count_2018.filter(regex='Q38').bfill(axis=1).iloc[:, 0].fillna('unknown')
fav_count_2018.sort_values(inplace=True)
fav_count_2018.drop(['Journal Publications', 'ArXiv & Preprints'], inplace=True)
fav_count_2018['Journal Publications'] = 5291.0
fav_count_2018.drop(['FiveThirtyEight.com', 'Cloud AI Adventures (YouTube)', 'KDnuggets Blog', 'DataTau News Aggregator', 'FastML Blog', 'Medium Blog Posts'], inplace=True)
fav_count_2018['Blogs'] = 11113.0

ax = pd.concat({'2019 Favorite Media':fav_count, '2018 Favorite Media': fav_count_2018[3:]}, axis=1).plot.barh(alpha=0.3, color=['g','b'])
ax.set_xlabel('No. of Votes')
ax.set_title('Favorite Media Sources in 2019 and 2018')
plt.show()

<h1 style="text-align:center;color:blue;">What exactly changed when it comes to tools used in 2018 vs. 2019?</h1>
<h2 style="text-align:center;color:blue;">Part II</h2>
<p style="text-align:center;">There is only few questions which allowed me to compare answers from 2018 to 2019. Taught be previous example I wanted to limit data selection as much as I can. I went through all of the schema and found few questions I deemed worth of comparing, especially that they give an insight into evolving data science landscape which matters not only for employees but also employers. This complets macro picture perfectly.</p>
<p style="text-align:center;">Datasets are 'comparable'. By this I mean that amount of responders in both is similar (19718 vs 23860). However, some answers seem to be skewed by different selection options or more selection options being available. Take that into consideration when looking at plots below.</p>

<h3 style="text-align:center;">Q: Does your current employer incorporate machine learning methods into their business?</h3>

<p style="text-align:center;">Answers between 2018 and 2019 are not so different, but those slight differences are very indicative. **This questions is most interesting in terms of 'negatives'**. Notice how many people less claims to 'Not know' if their company is using machine learning methods. Another thing which I noticed instantly is far bigger interest in exploring ML methods comparing to previous year and far smaller amount of people claiming not using any ML methods. Other options, describing companies which are already using 'some' ML pipeline **just show consistency across last year**, I read it as a sign that those methods are working fine in company environment and that's why no change is reflected in answer.</p>

In [ ]:
mltools = multiple_choice.filter(regex='Q8')
mltools_count = mltools.apply(lambda x: x.value_counts())
mltools_count.drop(mltools_count[12:].index, inplace=True)
mltools_count = mltools_count.filter(regex='Q8').bfill(axis=1).iloc[:, 0].fillna('unknown')
mltools_count.drop(['Does your current employer incorporate machine learning methods into their business?'], inplace=True)

mltools_2018 = multiple_choice_2018.filter(regex='Q10')
mltools_count_2018 = mltools_2018.apply(lambda x: x.value_counts())
mltools_count_2018.drop(mltools_count_2018[12:].index, inplace=True)
mltools_count_2018 = mltools_count_2018.filter(regex='Q10').bfill(axis=1).iloc[:, 0].fillna('unknown')
mltools_count_2018.drop(['Does your current employer incorporate machine learning methods into their business?'], inplace=True)


ax = pd.concat({'2019':mltools_count, '2018': mltools_count_2018}, axis=1).plot.barh(alpha=0.3, color=['g','b'])
ax.set_xlabel('No. of Votes')
ax.set_title('Does your current employer incorporate machine learning methods into their business?')
plt.show()


<h3 style="text-align:center;">Q: What activities make up an important part of your role at work?</h3>
<p style="text-align:center;">You can see that one question didn't make into Kaggle survey last year, this means that here we have count only for year 2019. It also explains why 2018 had more responses on each selection (dilution). I would say that divisions of labor are pretty consistent across last two years. Introducing question about participation in experimentation seemed to please a lot of people which in previous years were simply choosing 'Other' option. **I hesitate a statement that most of current work with ML in business environment is within this particular field.**</p>

In [ ]:
activities = multiple_choice.filter(regex='Q9')
activities_count = activities.apply(lambda x: x.value_counts())
activities_count.drop(activities_count[12:].index, inplace=True)
activities_count = activities_count.filter(regex='Q9').bfill(axis=1).iloc[:, 0].fillna('unknown')
activities_count.drop(['Select any activities that make up an important part of your role at work: (Select all that apply) - Selected Choice - Analyze and understand data to influence product or business decisions',
                       'Select any activities that make up an important part of your role at work: (Select all that apply) - Selected Choice - Build and/or run a machine learning service that operationally improves my product or workflows',
                       'Select any activities that make up an important part of your role at work: (Select all that apply) - Selected Choice - Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data',
                       'Select any activities that make up an important part of your role at work: (Select all that apply) - Selected Choice - Build prototypes to explore applying machine learning to new areas',
                       ], inplace=True)

activities_2018 = multiple_choice_2018.filter(regex='Q11')
activities_count_2018 = activities_2018.apply(lambda x: x.value_counts())
activities_count_2018.drop(activities_count_2018[12:].index, inplace=True)
activities_count_2018 = activities_count_2018.filter(regex='Q11').bfill(axis=1).iloc[:, 0].fillna('unknown')
activities_count_2018.drop(['Select any activities that make up an important part of your role at work: (Select all that apply) - Selected Choice - Analyze and understand data to influence product or business decisions',
                       'Select any activities that make up an important part of your role at work: (Select all that apply) - Selected Choice - Build and/or run a machine learning service that operationally improves my product or workflows',
                       'Select any activities that make up an important part of your role at work: (Select all that apply) - Selected Choice - Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data',
                       'Select any activities that make up an important part of your role at work: (Select all that apply) - Selected Choice - Build prototypes to explore applying machine learning to new areas',
                       'Select any activities that make up an important part of your role at work: (Select all that apply) - Selected Choice - Do research that advances the state of the art of machine learning',
                        ], inplace=True)

ax1 = pd.concat({'2019':activities_count, '2018': activities_count_2018}, axis=1).plot.barh(alpha=0.3, color=['g','b'])
ax1.set_xlabel('No. of Votes')
ax1.set_title('Activities that make up an important part of your role at work')
plt.show()

<h3 style="text-align:center;">Q: What is the primary tool that you use to analyze data?</h3>
<p style="text-align:center;">Again, answers are consistent with previous years. We actually see **only slight uptick in 'closed' end-user oriented software like Excel/Google Cloud **and move from local environment towards cloud based providers. This is in agreement with current narrative, we will probably see more migration towards cloud services.</p>

In [ ]:
primary_tool = multiple_choice.filter(regex='Q14')
primary_tool_count = primary_tool.apply(lambda x: x.value_counts())
primary_tool_count.drop(primary_tool_count[12:].index, inplace=True)
primary_tool_count = primary_tool_count.filter(regex='Q14').bfill(axis=1).iloc[:, 0].fillna('unknown')
primary_tool_count.drop(primary_tool_count[6:].index, inplace=True)

primary_tool_2018 = multiple_choice_2018.filter(regex='Q12')
primary_tool_count_2018 = primary_tool_2018.apply(lambda x: x.value_counts())
primary_tool_count_2018.drop(primary_tool_count_2018[12:].index, inplace=True)
primary_tool_count_2018 = primary_tool_count_2018.filter(regex='Q12').bfill(axis=1).iloc[:, 0].fillna('unknown')
primary_tool_count_2018.drop(primary_tool_count_2018[6:].index, inplace=True)
primary_tool_count_2018.rename(index={'Local or hosted development environments (RStudio, JupyterLab, etc.)':'Local development environments (RStudio, JupyterLab, etc.)'}, inplace=True)


ax2 = pd.concat({'2019':primary_tool_count, '2018': primary_tool_count_2018}, axis=1).plot.barh(alpha=0.3, color=['g','b'])
ax2.set_xlabel('No. of Votes')
ax2.set_title('What is the primary tool that you use to analyze data?')
plt.show()

<h1 style = "text-align:center;color:blue">Conclusion</h1>
<p style = "text-align:center;">One could explore survey such this in many ways. Biggest takeaway from what I've done here is not so surprising, it could be summarized as 'Things stay the same, and when they don't - they move slowly'. We don't observe anything earth shattering looking at the responses from community, it seems that ML is keeping it's pace of adaptation and motivating more and more people to start coding **(which may be one of it's biggest successes)**. I can safely say that after last few years of grace period, some companies feel more confident to deploy ML methods in their pipeline - which we also seen on a plot. Inequalities in macro scale are unrelated to changes in landscape of whole data science, they are connected with macroeconomic disparity and nothing else. Talented engineers are everywhere, it just seems that most lucky ones *(when it comes to compensation)* are in US, even when comparing to EU countries.</p>

<p style = "text-align:center;">I need to apologize for somehow chaotic writing style, very functional yet not optimized code and raw visualization, it's the best one can do in such a short time. However, I am very happy to actually finish my first kaggle competition ever! It was a sweat, no doubt.</p>